In [14]:
import binascii
import time
import os
import gc
import shutil
import numpy
from fobos import openadc, projmgr, pynqctrl, nexys3dut, scope
from fobos.analyzer import ttestOnline
from pynq import Xlnk
import numpy as np
from pynq import Overlay
import sys
WORKSPACE       = "/home/xilinx/fobosworkspace"
#WORKSPACE      = "/mnt/sda1/fobosworkspace"
PROJECT_NAME    ="kyber"
DIN_FILE_NAME   = "dinFile.txt"
CIPHER_FILE     = "do.txt"
TRACE_FILE      = "powerTraces.npy"
#DUT_BIT_FILE    = "half_duplex_dut.bit"
DUT_BIT_FILE    = "FOBOS_DUT.bit"
OUT_LEN         = 8# in 32-bit words
TRACE_NUM       = 5000
SAMPLING_FREQ   = 5
DUT_CLK         = 0.5
SAMPLE_NUM      = 1600000
#SAMPLE_NUM      = 2000

#Scope ########################################################################################################
scopConfig = {
         'OSCILLOSCOPE'       : 'AGILENT', #AGILENT|OPENADC
         'OSCILLOSCOPE_IP'    : '192.168.10.10',
         'OSCILLOSCOPE_PORT'  : '5025',
         'AUTOSCALE'          : 'NO',   # YES|NO    
         'IMPEDANCE'          : 'ONEMEG', #FIFTY|ONEMEG
         # VOLTAGE AND TIME RANGE OPTIONS        
         'CHANNEL1_RANGE'     : '0.050V',
         'CHANNEL2_RANGE'     : '6V',
         'CHANNEL3_RANGE'     : 'OFF', # ON|OFF|voltage range
         'CHANNEL4_RANGE'     : 'OFF', # ON|OFF|voltage range
         'TIME_RANGE'         :  '0.410000',
         'TIMEBASE_REF'       : 'LEFT',    
         # TRIGGER OPTIONS
         'TRIGGER_SOURCE'     : 'CHANNEL2',
         'TRIGGER_MODE'       : 'EDGE',   
         'TRIGGER_SWEEP'      : 'NORM',
         'TRIGGER_LEVEL'      : '0.5',
         'TRIGGER_SLOPE'      : 'POSITIVE',
         # ACQUIRE OPTIONS
         'ACQUIRE_TYPE'       : 'NORM', # NORM|PEAK|HRES|AVER
         'ACQUIRE_MODE'       : 'RTIM'   # RTIM | ETIM| SEG
}
scope = scope.Scope()
scope.setConfig(scopConfig)
print(scope.getConfig())
scope.openConnection()
scope.applyConfig()

{'OSCILLOSCOPE': 'AGILENT', 'OSCILLOSCOPE_IP': '192.168.10.10', 'OSCILLOSCOPE_PORT': '5025', 'RESOURCE': '', 'AUTOSCALE': 'NO', 'IMPEDANCE': 'ONEMEG', 'CHANNEL1_RANGE': '0.050V', 'CHANNEL2_RANGE': '6V', 'CHANNEL3_RANGE': 'OFF', 'CHANNEL_4RANGE': '', 'CHANNEL1_DISPLAY': '', 'CHANNEL2_DISPLAY': '', 'CHANNEL3_DISPLAY': '', 'CHANNEL4_DISPLAY': '', 'TIME_RANGE': '0.410000', 'TIMEBASE_REF': 'LEFT', 'TRIGGER_SOURCE': 'CHANNEL2', 'TRIGGER_MODE': 'EDGE', 'TRIGGER_SWEEP': 'NORM', 'TRIGGER_LEVEL': '0.5', 'TRIGGER_SLOPE': 'POSITIVE', 'SAMPLE_INPUT': '', 'ACQUIRE_TYPE': 'NORM', 'ACQUIRE_MODE': 'RTIM', 'ACQUIRE_COMPLETE': '', 'WAVE_DATA_SIZE': '', 'NUM_PWR_TRACE': '', 'SCREEN_CAP': '', 'SCREEN_FORMAT': '', 'SCREEN_NAME': '', 'OUTPUT_DIR': '', 'CHANNEL4_RANGE': 'OFF'}


In [15]:
overlay = Overlay("ctrl_top_wrapper.bit")


In [16]:
##########################
###configure worksapce
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(WORKSPACE)
pm.setProjName(PROJECT_NAME)
projDir = pm.getProjDir()
##########################
#configure controller
ctrl = pynqctrl.PYNQCtrl(overlay, inputSize = 2256, outputSize = 32)
ctrl.setDUTClk(DUT_CLK)
ctrl.setOutLen(OUT_LEN)
ctrl.setTriggerMode(pynqctrl.PYNQCtrl.TRG_FULL)
#ctrl.setTriggerLen(100000)
#ctrl.setTriggerWait(0)
ctrl.setDUTInterface(pynqctrl.PYNQCtrl.INTERFACE_8BIT)
##########################
#configure DUT
dut = nexys3dut.Nexys3DUT()
bitFile = os.path.join(projDir, DUT_BIT_FILE)
dut.setBitFile(bitFile)
dut.program()

Set clk frequency to 0.5
Programming device using the following commands:
sudo -u xilinx djtgcfg init -d Nexys3
sudo -u xilinx djtgcfg prog -d Nexys3 -i 0 -f /home/xilinx/fobosworkspace/kyber/FOBOS_DUT.bit
b'Programming device. Do not touch your board. This may take a few minutes...\nProgramming succeeded.\n'


In [17]:
##tvla
FVR_FILE = "fvrChoice.txt"
fvrFile = open(os.path.join(projDir, FVR_FILE), 'r')
ttestCalc = ttestOnline.TtestOnline(SAMPLE_NUM)

In [18]:

##########################
#setup i/o files
tvFileName = os.path.join(projDir, DIN_FILE_NAME)
tvFile = open(tvFileName, "r")
captureDir = pm.getCaptureDir()
cipherFileName = os.path.join(captureDir, CIPHER_FILE)
cipherFile = open(cipherFileName, "w")
traceFileName = os.path.join(captureDir, TRACE_FILE)
traceFile = open(traceFileName, "a+b")
#print("Copying test-vector file to attempt directory ...")
#shutil.copy(tvFileName, captureDir)
#########################
print("Loading test-vector file ...")
testVectors = tvFile.readlines()
print("Done loading.")
tic = time.time()
#traces = np.empty((TRACE_NUM, SAMPLE_NUM))
scope.arm()
time.sleep(1)
for traceNum in range(TRACE_NUM):
    time.sleep(0.1) #this delay seem to be needed
    result = ctrl.processData(testVectors[traceNum])
    #traces[traceNum, :] = trace
    cipherFile.write(result + '\n')
    if traceNum == 0:
        trigTrace = scope.readChannel('CHAN2')
        #print(trigTrace)
        start, end = scope.getCutPoints(trigTrace)
        print("Cutting trace parameters: start=%d, end = %d" % (start, end))
    trace = scope.readChannel('CHAN1')
    #print(trace.shape)
    #print(trace.shape)
    trace = trace[start:start + SAMPLE_NUM]
    #time.sleep(5)
    if traceNum != TRACE_NUM - 1: #after you finish do not arm scope. It will hang
        scope.arm()
    
    #numpy.save(traceFile, trace)
    c = fvrFile.read(1)
    if c == "0":
        #numpy.save(traceFile0, trace)
        ttestCalc.addTrace(trace, 0)
    else:
        ttestCalc.addTrace(trace, 1)
        #numpy.save(traceFile1, trace)
    if traceNum % 250 == 0 and traceNum != 0:
        collected = gc.collect() 
        print("Garbage collector: collected", "%d objects." % collected) 
        sys.stdout.write("Progress: {}%\r".format(traceNum/TRACE_NUM *100 ))
        sys.stdout.flush()
        ttestPlotFile = os.path.join(captureDir, "ttest{}".format(traceNum))
        ttestCalc.plot(ttestPlotFile, xTicksStep= 400000)
        
    #time.sleep(4)
#numpy.save(traceFile,traces)
#traces = None #Free memory
cipherFile.close()
tvFile.close()
traceFile.close()
    
toc = time.time()
print("Time elapsed = {} seconds.".format(toc - tic))

Successfully created new capture directory at /home/xilinx/fobosworkspace/kyber/capture/attempt-21
Loading test-vector file ...
Done loading.
Cutting trace parameters: start=6559, end = 1634600
Garbage collector: collected 4268 objects.
Garbage collector: collected 979 objects.
Garbage collector: collected 4841 objects.
Garbage collector: collected 4841 objects.
Garbage collector: collected 4841 objects.
Garbage collector: collected 4841 objects.
Garbage collector: collected 4841 objects.
Garbage collector: collected 4841 objects.


MemoryError: In draw_path: Out of memory

In [6]:
import matplotlib.pyplot as plt
f = open(traceFileName, 'rb')
print(traceFileName)
dataToPlot = numpy.load(f)
plt.plot(dataToPlot[0:10000])
#print(dataToPlot.shape)
#print(dataToPlot[10000:11000])
plt.show()

/home/xilinx/fobosworkspace/kyber/capture/attempt-48/powerTraces.npy


In [34]:
traces = None